# Solving the OCP via indirect approach, forced Lagrangian multi-pendulum on a cart

first order eqs:
$$
\dot{x} = v_x, \dot{y}
$$


In [ ]:
import sympy
import numpy as np
import function_definitions_forced as field_funcs
import helper_funcs as hfct
import pickle
import scipy
import scipy.optimize as opt
import OCP_construction_functions_forced as OCP_funcs
import ipdb
import matplotlib.pyplot as plt


In [ ]:
N_choice = 50
alpha_choice = 1
gamma_choice= 0.5
save_data_file = 'data/rigid_body/reference/ref_data.pkl'

OCP_parameters = {
            # Generic OCP params, always needed
            "T": 10.,   #s
            # "dim_q": 2,  # state dimension will be set by the control Lagrangian
            # "dim_u": 1,  # actuation dimension will be set by the control Lagrangian
            "N":N_choice,     #240, 160, 80, 40, 20
            "q0": sympy.Matrix([1,1,1e-2]),        #(m,1,1)
            "dq0": sympy.Matrix([0.,0.,0.]),       #(m/s,1/s,1/s)
            "qT": sympy.Matrix([sympy.pi/2,sympy.pi/2,sympy.pi/2]),      
            "dqT": sympy.Matrix([0.,0.,0.]),       #(vr,vphi1,vphi2)
            "r0" : [sympy.Matrix([1,0,0]),sympy.Matrix([0,1,0]),sympy.Matrix([0,0,1])],
            "rT" : [sympy.Matrix([0,0,-1]),sympy.Matrix([-1,0,0]),sympy.Matrix([0,1,0])],
            "alpha": alpha_choice,
            "gamma": gamma_choice,
            'folder_name': "rigid_body",
            #specific to model
            "I1":10.,
            "I2":9.,  # kg
            "I3":7.,
            "Aq": 1.237, #  kg/s^2
            "Adq": 12.77, # kg/s
            "A_u": 0.00052,
            "dim_u":2,
            "variable_names": ["phi", "theta","psi"]
        }
OCP_parameters["h"] = sympy.Symbol('h')
OCP_parameters["times"] = sympy.Matrix([i*OCP_parameters["h"] for i in range(OCP_parameters["N"]+1)])

In [ ]:
def coordinate_trafo_matrix(q):
    phi,theta,psi = q
    sinpsi = sympy.sin(psi)
    cospsi = sympy.cos(psi)
    sinphi = sympy.sin(phi)
    cosphi = sympy.cos(phi)
    sintheta = sympy.sin(theta)
    costheta = sympy.cos(theta)
    trafo_z_phi = sympy.Matrix([[cosphi,-sinphi,0],[sinphi,cosphi,0],[0,0,1]])
    trafo_z_psi = sympy.Matrix([[cospsi,-sinpsi,0],[sinpsi,cospsi,0],[0,0,1]])
    trafo_x_theta = sympy.Matrix([[1,0,0],[0,costheta,-sintheta],[0,sintheta,costheta]])
    trafo_mat= (trafo_z_phi@trafo_x_theta)@trafo_z_psi
    return sympy.Matrix(trafo_mat)

def body_trafo_matrix(q):
    phi,theta,psi = q
    sinpsi = sympy.sin(psi)
    cospsi = sympy.cos(psi)
    sinphi = sympy.sin(phi)
    cosphi = sympy.cos(phi)
    sintheta = sympy.sin(theta)
    costheta = sympy.cos(theta)
    trafo_mat = sympy.Matrix([[sinpsi*sintheta, cospsi,0],[cospsi*sintheta,-sinpsi,0],[costheta,0,1]])
    return trafo_mat


def rigid_body_lagrangian(q,vq,params):
    I1,I2,I3 = params['I1'],params['I2'],params['I3']
    I_mat = sympy.diag([I1,I2,I3],unpack=True)
    H_trafo = body_trafo_matrix(q)
    v_angular = sympy.Matrix(H_trafo@ vq)
    Lagrangian = 0.5* (v_angular.T)@(I_mat @ v_angular)
    return sympy.simplify(Lagrangian[0])


def f_L_rigid_body(q,vq,u,params):
    u0,u1=u 
    H_trafo = body_trafo_matrix(q)

    return H_trafo.T@sympy.Matrix([u0,u1,0])


def running_cost_rigid_body(q,u,params):
    A_u = params["A_u"]
    return A_u *u.transpose()@u/2

def mayer_term_rigid_body(q,v,params):
    q_qT = q - params["qT"]  
    vq_vqT = v - params["dqT"]  

    Phi = params["Aq"]*(q_qT.T @ q_qT)
    Phi+= params["Adq"]*(vq_vqT.T @vq_vqT)
    return Phi 

def mayer_term_rigid_body_new(q,v,params):
    # q_qT = q - params["qT"]  
    vq_vqT = v - params["dqT"]  
    trafo_mat=coordinate_trafo_matrix(q)
    r_0 = params['r0']
    r_T = params['rT']
    Phi = params["Adq"]*(vq_vqT.T @vq_vqT)
    for tmpr0,tmprT in zip(r_0,r_T):
        rT_from_q = trafo_mat @ tmpr0
        dr = rT_from_q - tmprT
        Phi += params["Aq"]*(dr.T @ dr)
    return Phi 

def g_mat_rigid_body(q,params):
    '''not really needed for forced Lagrangian'''
    return sympy.Matrix(sympy.eye(len(params["dim_u"])))


#  Initial guess from data

# Function definition

In [ ]:
continuous_eq = OCP_funcs.Direct_continuous_generator_forced_L_double_pendulum(OCP_parameters,Lagrangian=rigid_body_lagrangian
                                                               ,f_L=f_L_rigid_body
                                                               ,running_cost_func=running_cost_rigid_body
                                                               ,mayer_func=mayer_term_rigid_body_new
                                                               ,g_func=g_mat_rigid_body
                                                               ,I_func=None)

# initial guess creation

In [ ]:
OCP_parameters_np_version = hfct.sympy_to_np_dict(OCP_parameters,alpha_choice,gamma_choice)
OCP_parameters_np_version['r0'] = np.array(sympy.flatten(OCP_parameters_np_version['r0'] ),float).reshape([3,3])
OCP_parameters_np_version['rT'] = np.array(sympy.flatten(OCP_parameters_np_version['rT'] ),float).reshape([3,3])

q_d_use = np.linspace(OCP_parameters_np_version['q0'],OCP_parameters_np_version['qT'],len(OCP_parameters_np_version['times']))
lambda_d_use = 0.01*q_d_use
mu_use = [0.1,0.1,0.1]
nu_use = [0.1,0.1,0.1]
U_d_1_use = np.linspace([1.,1.],[-0.5,-0.5],len(OCP_parameters_np_version['times']))
U_d_2_use = np.linspace([1.,1.],[-0.5,-0.5],len(OCP_parameters_np_version['times']))


In [ ]:
OCP_parameters_np_version = hfct.sympy_to_np_dict(OCP_parameters,alpha_choice,gamma_choice)
parameters = OCP_parameters_np_version


with open(save_data_file, 'rb') as files:
    initial_guess_data = pickle.load(files)

q_d_start_guess = initial_guess_data["q_d_new"]
lam_d_start_guess = initial_guess_data["lambda_d_new"]
u_d_start_guess = initial_guess_data['u_d_new']

cs_q =  scipy.interpolate.CubicSpline(np.linspace(0,parameters["T"],len(q_d_start_guess)), q_d_start_guess.reshape([len(q_d_start_guess),3]))
cs_lam = scipy.interpolate.CubicSpline(np.linspace(0,parameters["T"],len(lam_d_start_guess)), lam_d_start_guess.reshape([len(lam_d_start_guess),3]))  
cs_u= scipy.interpolate.CubicSpline(np.linspace(0,parameters["T"],len(u_d_start_guess)),u_d_start_guess)

U_d_1_use = cs_u(np.array(parameters["times"]) + parameters["gamma"]*parameters["h"]).reshape(len(parameters["times"]),1,2)
U_d_2_use = cs_u(np.array(parameters["times"]) + (1-parameters["gamma"])*parameters["h"]).reshape(len(parameters["times"]),1,2)

q_d_use = cs_q(parameters["times"]).reshape(len(parameters["times"]),3,1)
lambda_d_use = cs_lam(parameters["times"]).reshape(len(parameters["times"]),3,1)
mu_use = initial_guess_data["mu_sol"] 
nu_use = initial_guess_data["nu_sol"] 
v_q_use =[]

import matplotlib.pyplot as plt
q_d_use = list(q_d_use)
lambda_d_use= list(lambda_d_use)
U_d_1_use = list(U_d_1_use)
U_d_2_use = list(U_d_2_use)
q_d_use = np.array(q_d_use)
lambda_d_use= np.array(lambda_d_use)
U_d_1_use = np.array(U_d_1_use)
U_d_2_use = np.array(U_d_2_use)



# Evolution via state eq

In [ ]:
discrete_equations = OCP_funcs.discrete_standard_direct_eq_generator_forced_L_double_pendulum(continuous_eq)

In [ ]:
import time
q_d_vec = [OCP_parameters_np_version["q0"]]
vq_d_vec = [OCP_parameters_np_version["dq0"]]# + np.array([0.01,0])]
dim_q_simu =len(OCP_parameters_np_version["q0"])
q_eqs = discrete_equations.k_eqs_description_q['state_eqs'] 
vq_eqs = discrete_equations.k_eqs_description_vq['state_eqs']
q_eqs = q_eqs.subs([["alpha",OCP_parameters_np_version["alpha"]],["gamma",OCP_parameters_np_version["gamma"]],["h",OCP_parameters_np_version["h"]]])
vq_eqs = vq_eqs.subs([["alpha",OCP_parameters_np_version["alpha"]],["gamma",OCP_parameters_np_version["gamma"]],["h",OCP_parameters_np_version["h"]]])

qk,vqk,_,_,u1k,u2k = discrete_equations.k_eqs_description_q["vars_k"]
qk1,vqk1,_,_,u1k1,u2k1 = discrete_equations.k_eqs_description_q["vars_k1"]
control_guess = np.linspace([-0.7,0.7],[0.7,-0.7],int(OCP_parameters_np_version['N'])) 
# control_guess = np.linspace([-0.4,0.4],[0.4,-0.4],int(OCP_parameters_np_version['N'])) 


control_guess= np.append(control_guess,np.zeros([OCP_parameters_np_version['N'],2]),axis=0)

k1solv_vec = sympy.flatten(qk1) + sympy.flatten(vqk1)
multiplier=1
# OCP_parameters_np_version["h"]= 0.25
for i in range(OCP_parameters_np_version['N']):
    # q_eqs_eval = q_eqs.subs([["alpha",OCP_parameters_np_version["alpha"]],["gamma",OCP_parameters_np_version["gamma"]],["h",OCP_parameters_np_version["h"]]])
    # vq_eqs_eval = vq_eqs.subs([["alpha",OCP_parameters_np_version["alpha"]],["gamma",OCP_parameters_np_version["gamma"]],["h",OCP_parameters_np_version["h"]]])
    q_eqs_eval = q_eqs.subs([[tmp1,tmp2] for tmp1, tmp2 in zip(qk,q_d_vec[-1])]).subs([[tmp1,tmp2] for tmp1, tmp2 in zip(vqk,vq_d_vec[-1])])
    vq_eqs_eval = vq_eqs.subs([[tmp1,tmp2] for tmp1, tmp2 in zip(qk,q_d_vec[-1])]).subs([[tmp1,tmp2] for tmp1, tmp2 in zip(vqk,vq_d_vec[-1])]).subs([[tmp1,tmp2]for tmp1,tmp2 in zip(u1k,control_guess[i])]).subs([[tmp1,tmp2]for tmp1,tmp2 in zip(u2k,control_guess[i])])
    lambdified_q = sympy.lambdify(k1solv_vec,q_eqs_eval)
    lambdified_vq = sympy.lambdify(k1solv_vec,vq_eqs_eval)
    generate_sol_vec = lambda x: np.hstack([lambdified_q(*x).flatten(),lambdified_vq(*x).flatten()])
    eq_sol = opt.root(generate_sol_vec,x0=np.hstack([q_d_vec[-1].flatten(),vq_d_vec[-1]]),method="lm")
    q_d_vec.append(eq_sol.x[:dim_q_simu])
    vq_d_vec.append(eq_sol.x[dim_q_simu:])
q_d_vec = np.array(q_d_vec)
vq_d_vec = np.array(vq_d_vec)
times = np.linspace(0,OCP_parameters_np_version["h"]*len(q_d_vec),len(q_d_vec))
plt.plot(times,q_d_vec.transpose()[0],label=r'$\phi$')
plt.plot(times,q_d_vec.transpose()[1],label=r'$\theta$')
plt.plot(times,q_d_vec.transpose()[2],label=r'$\psi$')
plt.legend()
plt.xlabel('time')
plt.ylabel('r')

In [ ]:
# plt.plot(times,vq_d_vec.transpose()[0],label=r'$\phi$')
# plt.plot(times,vq_d_vec.transpose()[1],label=r'$\theta$')
# plt.plot(times,vq_d_vec.transpose()[2],label=r'$\psi$')
# # plt.plot(times, vq_d_vec.transpose()[0]+vq_d_vec.transpose()[2],'--')
# plt.legend()
# plt.xlabel('time')
# plt.ylabel('v')


In [ ]:
basis_vects = [[np.array([1,0,0])],[np.array([0,1,0])],[np.array([0,0,1])]]

for tmpq in q_d_vec:
    trafo_mat = coordinate_trafo_matrix(tmpq)
    basis_vects[0].append(trafo_mat@basis_vects[0][0])
    basis_vects[1].append(trafo_mat@basis_vects[1][0])
    basis_vects[2].append(trafo_mat@basis_vects[2][0])

basis_vects = np.array(basis_vects,float)

basis_vects_sympy = [sympy.Matrix(basis_vects[0,-1]),sympy.Matrix(basis_vects[1,-1]),sympy.Matrix(basis_vects[2,-1])]

In [ ]:
# plt.plot(times,basis_vects[0].T[0][1:],'x-')
# plt.plot(times,basis_vects[0].T[1][1:],'x-')


In [ ]:
import time
start_k =0
end_k = start_k+49
# plt.plot(times[start_k:end_k],basis_vects[0].T[0][start_k:end_k],'x-')
# plt.plot(times[start_k:end_k],basis_vects[0].T[2][start_k:end_k],'x-')
plt.show()
ax = plt.figure().add_subplot(projection='3d')
for k_val in range(start_k,end_k,4):
    ax.quiver(0,0,0,*(basis_vects[0][k_val]),length=0.051,normalize=True,color='blue')
    ax.quiver(0,0,0,*(basis_vects[1][k_val]),length=0.051,normalize=True,color='green')
    ax.quiver(0,0,0,*(basis_vects[2][k_val]),length=0.051,normalize=True,color='orange')
    time.sleep(0.1)
k_val = -1
ax.quiver(0,0,0,*(basis_vects[0][k_val]),length=0.051,normalize=True,color='blue')
ax.quiver(0,0,0,*(basis_vects[1][k_val]),length=0.051,normalize=True,color='green')
ax.quiver(0,0,0,*(basis_vects[2][k_val]),length=0.051,normalize=True,color='orange')    
for tmp,tmpc in zip(OCP_parameters_np_version['rT'],['red','black','yellow']):
    ax.quiver(0,0,0,*(tmp),length=0.051,normalize=True,color=tmpc)    
ax.view_init(elev=10,azim=115,roll=0)


In [ ]:
OCP_parameters['rT'] = basis_vects_sympy
OCP_parameters_np_version['rT'] = basis_vects[:,-1]
# lambda_d_use = 0.01*q_d_vec

continuous_eq = OCP_funcs.Direct_continuous_generator_forced_L_double_pendulum(OCP_parameters,Lagrangian=rigid_body_lagrangian
                                                               ,f_L=f_L_rigid_body
                                                               ,running_cost_func=running_cost_rigid_body
                                                               ,mayer_func=mayer_term_rigid_body_new
                                                               ,g_func=g_mat_rigid_body
                                                               ,I_func=None)

discrete_equations = OCP_funcs.discrete_standard_direct_eq_generator_forced_L_double_pendulum(continuous_eq)



In [ ]:
plt.plot(q_d_vec.transpose()[0])
plt.plot(q_d_vec.transpose()[1])
plt.plot(q_d_vec.transpose()[2])

plt.xlabel('steps')
plt.ylabel('r')

# lambdify the equations for root finding

In [ ]:
#KKT direct approach with new Lagrangian
standard_direct_KKT_new = discrete_equations.calc_KKT_new(OCP_parameters_np_version)

lambdified_KKT_new =  sympy.lambdify(standard_direct_KKT_new[1],standard_direct_KKT_new[0].subs(discrete_equations.h,OCP_parameters_np_version["h"]).subs(discrete_equations.cont_equations.parameters["alpha"],alpha_choice).subs(discrete_equations.cont_equations.parameters["gamma"],gamma_choice))

lambdified_KKT_new_eval = lambda x :lambdified_KKT_new(*x)


# Executing root for solving KKT

In [ ]:
initial_guess_new = list(np.concatenate([mu_use,nu_use])) #mu and nu

for tmp in q_d_use: #q_d_use
    initial_guess_new+=list(tmp.flatten())
    #vq guess
#lambda terms
for tmp in lambda_d_use:
    initial_guess_new += list(tmp.flatten())

initial_guess_new += list(U_d_1_use.flatten()) 
initial_guess_new += list(U_d_2_use.flatten()) 

# initial_guess_new += list(control_guess[:OCP_parameters['N']+1].flatten()) 
# initial_guess_new += list(control_guess[:OCP_parameters['N']+1].flatten()) 


In [ ]:
#new lagrangian approach
standard_result_polar_KKT_new = opt.root(lambdified_KKT_new_eval,x0=initial_guess_new,method="lm")
# standard_result_polar_KKT_new = opt.root(lambdified_KKT_new_eval,x0=standard_result_polar_KKT_new.x,method="lm")


In [ ]:
standard_result_polar_KKT_new = opt.root(lambdified_KKT_new_eval,x0=standard_result_polar_KKT_new.x)


In [ ]:
standard_result_polar_KKT_new = opt.root(lambdified_KKT_new_eval,x0=standard_result_polar_KKT_new.x,method="lm")


In [ ]:
standard_result_polar_KKT_new

# Post-processing

In [ ]:
reshape_params = discrete_equations.cont_equations.parameters
reshape_N = reshape_params["N"]
reshape_dim_q = reshape_params["dim_q"]
reshape_dim_u = reshape_params["dim_u"]
mu_KKT_new,nu_KKT_new = standard_result_polar_KKT_new.x[:2*reshape_dim_q].reshape(2,reshape_dim_q)
q_d_KKT_new = standard_result_polar_KKT_new.x[2*reshape_dim_q:2*reshape_dim_q + reshape_dim_q * (reshape_N+1)].reshape(reshape_N + 1,reshape_dim_q)
lam_d_KKT_new = standard_result_polar_KKT_new.x[2*reshape_dim_q + reshape_dim_q * (reshape_N+1):2*reshape_dim_q + 2*reshape_dim_q * (reshape_N+1)].reshape(reshape_N + 1,reshape_dim_q)

U1_d_KKT_new = standard_result_polar_KKT_new.x[2*reshape_dim_q + 2*reshape_dim_q * (reshape_N+1):2*reshape_dim_q + 2*reshape_dim_q * (reshape_N+1) + (reshape_N+1)*reshape_dim_u].reshape(reshape_N + 1,reshape_params["dim_u"])
U2_d_KKT_new = standard_result_polar_KKT_new.x[2*reshape_dim_q + 2*reshape_dim_q * (reshape_N+1) + (reshape_N+1)*reshape_dim_u:].reshape(reshape_N + 1,reshape_params["dim_u"])

p_y_d_new = np.array([sympy.flatten(tmp) for tmp in discrete_equations.p_v_d_from_y_d(q_d_KKT_new,lam_d_KKT_new,U1_d_KKT_new,U2_d_KKT_new,OCP_parameters_np_version)])

v_y_d_new = []
for tmp1,tmp2,tmp3 in zip(q_d_KKT_new,lam_d_KKT_new,p_y_d_new):
    v_y_d_new.append(sympy.flatten(discrete_equations.cont_equations.calc_vy_fromq_p_new(tmp1,tmp2,tmp3,OCP_parameters_np_version)))
v_y_d_new = np.array(v_y_d_new) 


u_vec_KKT_new = []
for i in range(len(q_d_KKT_new)):
    u_vec_KKT_new.append(discrete_equations.cont_equations.u_expr_from_new_lambdified(*q_d_KKT_new[i],*lam_d_KKT_new[i],*v_y_d_new[i]).flatten())

u_vec_KKT_new = np.array(u_vec_KKT_new)

H_control_KKT_new = []
for i in range(len(q_d_KKT_new)):
    H_control_KKT_new.append(discrete_equations.cont_equations.new_control_H_v_calc(q_d_KKT_new[i],lam_d_KKT_new[i],v_y_d_new[i],u_vec_KKT_new[i],OCP_parameters_np_version))
H_control_KKT_new = np.array(H_control_KKT_new)  

# I_d_evo_new = np.array(p_y_d_new.transpose()[0],float)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(q_d_KKT_new.T[0])
plt.plot(q_d_KKT_new.T[1])
plt.plot(q_d_KKT_new.T[2])
plt.show()
plt.plot(u_vec_KKT_new.T[0])
plt.plot(u_vec_KKT_new.T[1])
plt.plot(control_guess.T[0,:50])
plt.plot(control_guess.T[1,:50])
# plt.plot(U1_d_KKT_new.T[1],'--')
plt.show()

plt.plot(v_y_d_new.T[0])
plt.plot(v_y_d_new.T[1])
plt.plot(v_y_d_new.T[2])

basis_evo = [[np.array([1,0,0])],[np.array([0,1,0])],[np.array([0,0,1])]]

for tmpq in q_d_vec:
    trafo_mat = coordinate_trafo_matrix(tmpq)
    basis_evo[0].append(trafo_mat@basis_evo[0][0])
    basis_evo[1].append(trafo_mat@basis_evo[1][0])
    basis_evo[2].append(trafo_mat@basis_evo[2][0])
plt.show()
basis_evo= np.array(basis_evo)
plt.plot(basis_evo[0].T[0])
plt.plot(basis_evo[0].T[1])
plt.plot(basis_evo[0].T[2])
plt.hlines(OCP_parameters_np_version['rT'][0],0,50)

# plot results

In [ ]:
q_d_new_KKT_sol_cart = []

q_d_new_KKT_sol_cart = q_d_KKT_new    
q_d_new_KKT_sol_cart = np.array(q_d_new_KKT_sol_cart)


plt.plot(OCP_parameters_np_version["times"],q_d_new_KKT_sol_cart.transpose()[0],'--',label='new x evo')
plt.plot(OCP_parameters_np_version["times"],q_d_new_KKT_sol_cart.transpose()[1],'--',label='new theta1 evo')
plt.plot(OCP_parameters_np_version["times"],q_d_new_KKT_sol_cart.transpose()[2],'--',label='new theta2 evo')
plt.legend()
plt.xlabel('t')
plt.ylabel('y')
plt.tight_layout()

plt.show()

plt.plot(OCP_parameters_np_version["times"],lam_d_KKT_new.transpose()[0],'--',label='new approach lam_r')
plt.legend()
plt.xlabel('t')
plt.ylabel('lambda_r')
plt.tight_layout()


plt.show()
plt.plot(OCP_parameters_np_version["times"],lam_d_KKT_new.transpose()[1],'--',label='new aproach lam_phi')
plt.legend()
plt.xlabel('t')
plt.ylabel('lambda_phi')
plt.tight_layout()



plt.show()
plt.plot(U1_d_KKT_new[:-1].transpose()[0],'x',label='new u0')
plt.plot(U1_d_KKT_new[:-1].transpose()[1],'x',label='new u1')
# plt.plot(U1_d_KKT_new[:-1].transpose()[2],'x',label='new u2')
plt.plot(U_d_1_use[:-1,0,0],'--',label='comparison1')
plt.plot(U_d_1_use[:-1,0,1],'--',label='comparison2')
# plt.plot(U_d_1_use[:-1,0,2],'--',label='comparison3')
plt.legend()
plt.xlabel('N step')
plt.ylabel('U_1')
plt.tight_layout()
plt.show()

plt.plot(U2_d_KKT_new[:-1],'x',label='new')
plt.plot(U_d_2_use[:-1,0,0],'--',label='comparison1')
plt.plot(U_d_2_use[:-1,0,1],'--',label='comparison2')
# plt.plot(U_d_2_use[:-1,0,2],'--',label='comparison3')
plt.legend()
plt.xlabel('N step')
plt.ylabel('U_2')
plt.tight_layout()


plt.show()
plt.plot(U_d_2_use[:-1,0,0],'--',label='comparison')
plt.plot(u_vec_KKT_new[:,0],label='new u0d')
plt.plot(u_vec_KKT_new[:,1],label='new u1d')
# plt.plot(u_vec_KKT_new[:,2],label='new u2d')
plt.xlabel('N step')
plt.ylabel('u_d')
plt.legend()

plt.show()
plt.plot((np.array(I_d_evo_new)-np.array(I_d_evo_new)[0]),'X-',label="I new")
plt.legend()
plt.xlabel('N step')
plt.ylabel('I')
plt.tight_layout()

plt.show()
plt.plot((H_control_KKT_new[:-1]),'-*',label='new control H')
plt.legend()


# Saving data

In [ ]:
from pathlib import Path 

if not standard_result_polar_KKT_new.success:
    print('did not find a solution in control dependent case, not storing the result')
else:
    storage_dict = dict()
    storage_dict["parameters"] = OCP_parameters_np_version
    storage_dict["q_d_new"] = q_d_KKT_new
    storage_dict["q_d_new_cartesian"] = q_d_new_KKT_sol_cart
    storage_dict["lambda_d_new"] = lam_d_KKT_new
    storage_dict["p_y_d_new"] = np.array(p_y_d_new,dtype=float)
    storage_dict["U1_d_new"] = np.array(U1_d_KKT_new,dtype=float)
    storage_dict["U2_d_new"] = np.array(U2_d_KKT_new,dtype=float)
    storage_dict["u_d_new"] = np.array(u_vec_KKT_new,dtype=float)
    # storage_dict["I_d_new"] = np.array(I_d_evo_new,dtype=float)
    storage_dict["H_control_new"] = np.array(H_control_KKT_new,dtype=float)
    storage_dict["v_y_d_new"] = np.array(v_y_d_new,dtype=float)
    storage_dict["mu_sol"] = np.array(mu_KKT_new,dtype=float)
    storage_dict["nu_sol"] = np.array(nu_KKT_new,dtype=float)
    storage_dict['basis_rotation_evo'] = basis_evo


    dirpath = "data/" + OCP_parameters_np_version["folder_name"]+"/data_"+"a=" + str(OCP_parameters_np_version["alpha"])  +"g=" + str(OCP_parameters_np_version["gamma"])
    Path(dirpath).mkdir(parents=True, exist_ok=True)

    file_name = dirpath+"/data_fixed" +"a=" + str(OCP_parameters_np_version["alpha"])  +"g=" + str(OCP_parameters_np_version["gamma"])  +"N=" + str(OCP_parameters_np_version["N"]) + ".pkl"
    with open(file_name, 'wb') as ffile:
        pickle.dump(storage_dict, ffile)
        ffile.close()